In [1]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import datetime

In [6]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

In [7]:
start_year = 1989
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['PREC_ACC_NC'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['PREC_ACC_NC'].dims[1:], coords={'south_north': ds['PREC_ACC_NC'].coords['south_north'], 'west_east': ds['PREC_ACC_NC'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s.PREC_ACC_NC.values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'p': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-10-06 17:08:57
2024-10-06 17:20:03
2024-10-06 17:32:40
2024-10-06 17:46:17
2024-10-06 18:01:38
2024-10-06 18:14:24
2024-10-06 18:28:09
2024-10-06 18:41:45
2024-10-06 18:55:11
2024-10-06 19:08:37
2024-10-06 19:22:38
2024-10-06 19:37:01
2024-10-06 19:50:31
2024-10-06 20:05:33
2024-10-06 20:20:16
2024-10-06 20:34:59
2024-10-06 20:49:34
2024-10-06 21:03:05
2024-10-06 21:18:27
2024-10-06 21:33:20
2024-10-06 21:48:47
2024-10-06 22:03:40
2024-10-06 22:20:39
2024-10-06 22:32:05


In [2]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 1989
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.T2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['T2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['T2'].dims[1:], coords={'south_north': ds['T2'].coords['south_north'], 'west_east': ds['T2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['T2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'t2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'T2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-02 20:39:32
2024-11-02 20:50:45
2024-11-02 21:04:29
2024-11-02 21:17:42
2024-11-02 21:31:51
2024-11-02 21:44:50
2024-11-02 21:58:38
2024-11-02 22:11:29
2024-11-02 22:24:44
2024-11-02 22:39:08
2024-11-02 22:53:16
2024-11-02 23:07:33
2024-11-02 23:21:28
2024-11-02 23:35:33
2024-11-02 23:48:10
2024-11-03 00:01:44
2024-11-03 00:14:54
2024-11-03 00:27:22
2024-11-03 00:40:50
2024-11-03 00:54:04
2024-11-03 01:07:36
2024-11-03 01:20:55
2024-11-03 01:34:12
2024-11-03 01:48:56


In [3]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 1989
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)

    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:

            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  
            else:
                num_days = 28  

        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.TD2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)

            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['TD2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['TD2'].dims[1:], coords={'south_north': ds['TD2'].coords['south_north'], 'west_east': ds['TD2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['TD2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'td2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'TD2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-03 01:03:05
2024-11-03 01:16:37
2024-11-03 01:30:18
2024-11-03 01:44:36
2024-11-03 01:58:41
2024-11-03 02:12:29
2024-11-03 02:26:18
2024-11-03 02:39:08
2024-11-03 02:52:16
2024-11-03 03:05:38
2024-11-03 03:19:17
2024-11-03 03:33:23
2024-11-03 03:46:27
2024-11-03 04:00:33
2024-11-03 04:13:32
2024-11-03 04:27:24
2024-11-03 04:41:06
2024-11-03 04:53:11
2024-11-03 05:07:01
2024-11-03 05:20:07
2024-11-03 05:33:11
2024-11-03 05:45:51
2024-11-03 05:58:38
2024-11-03 06:11:39


In [2]:
import glob
def create_temp_dataset(arr_t, latt, lonn):

    arr_ntt = arr_t.reshape(43, 92, 24, arr_t.shape[1], arr_t.shape[2])
    selected_data = np.concatenate((arr_ntt[:, :, 0:6, :, :], arr_ntt[:, :, 18:24, :, :]), axis=2)

    arr_tmin = np.nanmean(np.nanmin(selected_data, axis=2),axis=1)
    arr_tmean = np.nanmean(np.nanmean(selected_data, axis=2),axis=1)
    
    ds_tmin = xr.Dataset(
        {'t': (['year',  'lat', 'lon'], arr_tmin)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )
    
    ds_tmean = xr.Dataset(
        {'t': (['year',  'lat', 'lon'], arr_tmean)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )

    return ds_tmin,ds_tmean



def create_dtemp_dataset(arr_t, latt, lonn):

    arr_ntt = arr_t.reshape(43, 92, 24, arr_t.shape[1], arr_t.shape[2])
    selected_data = np.concatenate((arr_ntt[:, :, 0:6, :, :], arr_ntt[:, :, 18:24, :, :]), axis=2)

    arr_tmin = np.nanmean(np.nanmin(selected_data, axis=2),axis=1)
    arr_tmean = np.nanmean(np.nanmean(selected_data, axis=2),axis=1)
    
    ds_dtmin = xr.Dataset(
        {'dt': (['year',  'lat', 'lon'], arr_tmin)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )
    
    ds_dtmean = xr.Dataset(
        {'dt': (['year',  'lat', 'lon'], arr_tmean)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )

    return ds_dtmin,ds_dtmean

In [3]:
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern = 'TD2.wrf2d_d01_????-??-??.nc'

folder_names = [

    'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',

]


for folder in folder_names:
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    full_path = os.path.join(base_path, folder, file_pattern)
    all_files = glob.glob(full_path)
    #####
    summer_files = [f for f in all_files if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_summer = xr.open_mfdataset(summer_files)
    ds_jja = ds_summer.sel(time=ds_summer['time'].dt.month.isin([6, 7, 8]))
    lonn = ds_jja.lon.values
    latt = ds_jja.lat.values
    arr_t = ds_jja.td2.values

    ds_dtmin,ds_dtmean = create_dtemp_dataset(arr_t, latt, lonn)

    output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'
    ds_dtmin.to_netcdf(output_folder+'dtemp_min_'+folder+'.nc')
    ds_dtmean.to_netcdf(output_folder+'dtemp_mean_'+folder+'.nc')

2024-11-03 21:16:09
2024-11-03 21:21:10
2024-11-03 21:26:20
2024-11-03 21:31:39
2024-11-03 21:36:54
2024-11-03 21:41:56
2024-11-03 21:47:04
2024-11-03 21:52:31
2024-11-03 21:58:03
